In [55]:
import json
import numpy as np

H3 = 'h3'
LI = 'li'
A = 'a'

def link_is_not_date(link):
        months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
        return all([-1 == link.find(month) for month in months])

def tag_not_empty(tag):
        return len(tag) > 0

def tag_type(tag):
        if 'li' in tag:
                return LI
        elif 'h3' in tag:
                return H3
        elif 'a' in tag:
                return A
                
def parse_json(json_file_name:str, stop_year:str):
        #Nessesary to remove links to dates which almost all events have

        with open(json_file_name, 'r') as f:
                tags = np.array(json.load(f))

        events = [] #List to save indicies on the form (YEAR, [...,EVENTS THAT YEAR,...])
        curr_year_events = [] #List to save all events ONE year on the form (EVENT_TITLE, [..., WIKI LINKS FOR EVENT, ...])

        #Event state for in iteration
        curr_year = None
        curr_event = None
        curr_links = []

        for tag in tags:
                type = tag_type(tag)
                #Nessesary for skipping html tags before the list of historical events
                if not curr_year and type == H3:
                        curr_year = tag[H3][0:4]
                        continue
                #Save previous year's state to "events" and reset state for next years events
                if type == H3:
                        if curr_year == stop_year:
                                events.append((curr_year,curr_year_events[:-1])) 
                                break
                        events.append((curr_year,curr_year_events)) 
                        curr_year = tag[H3][0:4]
                        curr_year_events = []

                #Add event to the current year (curr_year)
                elif type == LI:
                        if curr_event != None:
                                curr_year_events.append((curr_event, curr_links))
                                curr_links, curr_event = [], None
                        format = tag[LI].split(':')
                        if len(format) == 2:
                                curr_event = format[1]
                
                #Add link to the current iteration of event (curr_event)       
                elif type == A:
                        if curr_event != None:
                                link = tag[A]
                                if tag_not_empty(link) and link_is_not_date(link):
                                        curr_links.append(link)
        return events

In [56]:
parse_json("timeline20", '2000')
#parse_json("timeline21", '2017')

[('1901',
  [(' The Australian colonies federate.',
    ['/wiki/Australian_Colonies', '/wiki/Federation_of_Australia']),
   (' Edward VII becomes King of the United Kingdom and the British Dominions and Emperor of India upon the death of Queen Victoria.',
    ['/wiki/Edward_VII', '/wiki/Queen_Victoria']),
   (' Platt Amendment limits the autonomy of Cuba in exchange for withdrawal of American troops.',
    ['/wiki/March_2', '/wiki/Platt_Amendment', '/wiki/Cuba']),
   (' Emily Hobhouse reports on the terrible conditions in the 45 British concentration camps for Boer women and children in South Africa.',
    ['/wiki/Emily_Hobhouse#Conditions_in_the_camps',
     '/wiki/British_concentration_camps',
     '/wiki/Second_Boer_War',
     '/wiki/South_Africa']),
   (" Assassination of William McKinley. Vice President Theodore Roosevelt assumes office as President of the United States following McKinley's death on September 14.",
    ['/wiki/Assassination_of_William_McKinley',
     '/wiki/Theodo